<a href="https://colab.research.google.com/github/Nagarajmalkhed/GSTBILLING-/blob/main/TEXT_TO_SPEECH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchaudio


In [2]:
pip install unidecode inflect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 12.0 MB/s eta 0:00:00


In [5]:
import torch
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import Audio # For playing audio in Jupyter/Colab




In [9]:
# 1. Load the Pre-trained TTS Pipeline Bundle
# TACOTRON2_WAVERNN_CHAR_LJSPEECH is a bundle trained on the LJSpeech dataset
bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_CHAR_LJSPEECH
processor = bundle.get_text_processor()
tacotron2 = bundle.get_tacotron2()
vocoder = bundle.get_vocoder()



In [17]:
text = "ACTE TECH by Facebook's AI Research lab."


In [10]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tacotron2.to(device).eval()
vocoder.to(device).eval()

_WaveRNNVocoder(
  (_model): WaveRNN(
    (upsample): UpsampleNetwork(
      (resnet): MelResNet(
        (melresnet_model): Sequential(
          (0): Conv1d(80, 128, kernel_size=(5,), stride=(1,), bias=False)
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): ResBlock(
            (resblock_model): Sequential(
              (0): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
              (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
              (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
              (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (4): ResBlock(
            (resblock_model): Sequential(
              (0): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
              (1): Batch

In [12]:
with torch.inference_mode():
    # Process text into a sequence of token IDs
    processed_tokens, lengths = processor(text)
    processed_tokens = processed_tokens.to(device)
    lengths = lengths.to(device)

    # Generate Mel-Spectrogram using Tacotron 2
    # spec: Mel-Spectrogram tensor
    spec, spec_lengths, _ = tacotron2.infer(processed_tokens, lengths)


In [13]:
with torch.inference_mode():
    # Convert the Mel-Spectrogram to a raw audio waveform
    waveform, lengths = vocoder(spec, spec_lengths)


In [19]:
sample_rate = vocoder.sample_rate

# Save to file
torchaudio.save("synthesized_speech.wav", waveform[0:1].cpu(), sample_rate=sample_rate)
print(f"Synthesized speech saved to synthesized_speech.wav at {sample_rate} Hz.")

# Optionally, play in notebook environment
Audio(waveform[0:1].cpu(), rate=sample_rate)

# Visualization (optional)
# plt.figure(figsize=(10, 4))
# plt.imshow(spec[0].cpu().detach(), origin='lower', aspect='auto')
# plt.title("Generated Mel-Spectrogram")
# plt.show()

Synthesized speech saved to synthesized_speech.wav at 22050 Hz.


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:247: UserWarning: torio.io._streaming_media_encoder.StreamingMediaEncoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s 